In [2]:
import numpy as np
import pandas as pd
from mouselabdiscrete import NewMouselabEnv
from evaluation import *
from distributions import PiecewiseUniform, expectation
from time import time
import pickle
import json

from joblib import Parallel, delayed
import multiprocessing
num_cores = 1


/home/aashay/virtualenvs/mouselab/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [3]:
gambles = 7
attributes = 4
alpha = 0.15 #dispersion
num_samples = 10
cost=2

intervals = [(-100,-80),(-80,-60),(-60,-40),(-40,-20),(-20,0),(0,20),(20,40),(40,60),(60,80),(80,100)]

In [4]:
def reshape_payoffs(l, n = 7):
    #return [l[i:i+n] for i in range(0, len(l), n)]
    return list(np.array(l).reshape(n, len(l)//n).T)

for participant_id in range(43):
    for stock_id in range(2):
        dist_file_name = "bias_dists/t_dist_" + str(participant_id) + '.pkl'
        out_file_name = "true_samples/" + 'true' + str(participant_id) + '_' + str(stock_id) + '.json'
        true_dist = np.array(pickle.load(open(dist_file_name, 'rb'))[stock_id])
        
        reward = PiecewiseUniform(intervals, true_dist)
        
        env_array = []

        def make_samples(num_samples = num_samples):
            np.random.seed(participant_id*2 + stock_id)
            for i in range(num_samples):
                env = NewMouselabEnv(gambles, attributes, reward, cost, alpha = alpha, sample_term_reward = True)
                env_array.append(env)

        make_samples()   

        def save_samples():

            def run_episode(env):
            #for env in test_env_array:
                #print(env.ground_truth)
                env._reset()
                return (list(env.ground_truth), list(env.distRV.ground_truth))

            info_list = Parallel(n_jobs=num_cores)(delayed(run_episode)(env) for env in env_array)

            df = pd.DataFrame(info_list, columns = ['payoffs', 'probabilities'])
            #df.to_csv(csv_file_name, index = False)
            
            #df['payoffs'] = df['payoffs'].apply(eval)
            df['payoffs'] = df['payoffs'].apply(reshape_payoffs)
            #df['probabilities'] = df['probabilities'].apply(eval)
            df.rename(columns = {'payoffs':'payoff_matrix'}, inplace = True)
            env_list = df.to_json(orient = "records", double_precision = 10)
            #np.save(reward_file_name, reward_list)
            
            a = {"params":{"alpha" : alpha}, "standard" : eval(env_list)}
            with open(out_file_name, 'w') as json_file:
              json.dump(a, json_file)
        save_samples()

In [1]:
for participant_id in range(43):
    for stock_id in range(2):
        print(participant_id*2 + stock_id)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
